In [ ]:
# J. Ryu, Electron Microscopy and Spectroscopy Lab., Seoul National University
import hyperspy.api as hys
import numpy as np
from scipy import ndimage
import matplotlib.pyplot as plt
import ipywidgets as pyw
import tkinter.filedialog as tkf

In [ ]:
%matplotlib inline

In [ ]:
eels_adr = tkf.askopenfilename()
print(eels_adr)

In [ ]:
e_start = 1.0
e_end = 5.0
e_step = 0.01
energy_range = np.arange(e_start, e_end, e_step)
num_energy = len(energy_range)
print(num_energy)

In [ ]:
eelsi = hys.load(eels_adr, signal_type="EELS")
print(eelsi)

In [ ]:
# binning
cluster_size_y = 2
cluster_size_x = 2
stride_y = 2
stride_x = 2
eelsi_binned = []
height = len(range(0, eelsi.data.shape[0]-cluster_size_y+1, stride_y))
width = len(range(0, eelsi.data.shape[1]-cluster_size_x+1, stride_x))
for i in range(0, eelsi.data.shape[0]-cluster_size_y+1, stride_y):
    for j in range(0, eelsi.data.shape[1]-cluster_size_x+1, stride_x):
        eelsi_binned.append(np.sum(eelsi.data[i:i+cluster_size_y, j:j+cluster_size_x], axis=(0, 1)))
        
eelsi_binned = np.asarray(eelsi_binned).reshape(height, width, -1)
eelsi_binned = hys.signals.Signal1D(eelsi_binned)
eelsi_binned.set_signal_type("EELS")
eelsi_binned.metadata = eelsi.metadata
eelsi_binned.axes_manager = eelsi.axes_manager
eelsi_binned.axes_manager[0].size = width
eelsi_binned.axes_manager[1].size = height
eelsi_binned.axes_manager[0].scale = eelsi.axes_manager[0].scale * cluster_size_x
eelsi_binned.axes_manager[1].scale = eelsi.axes_manager[1].scale * cluster_size_y
print(eelsi_binned.axes_manager)

In [ ]:
eelsi_cropped = eelsi_binned.isig[e_start:e_end].data
print(eelsi_cropped.shape)

In [ ]:
# line by line horizontally
eels_lines = hys.signals.Signal1D(np.sum(eelsi_cropped[:, :90], axis=1))
eels_lines.set_signal_type("EELS")
eels_lines.axes_manager[1].scale = e_step
eels_lines.axes_manager[1].offset = e_start
eels_lines.axes_manager[1].units = "eV"
print(eels_lines)
print(eels_lines.axes_manager)

In [ ]:
# line by line vertically
eels_lines = hys.signals.Signal1D(np.sum(eelsi_cropped, axis=0))
eels_lines.set_signal_type("EELS")
eels_lines.axes_manager[1].scale = e_step
eels_lines.axes_manager[1].offset = e_start
eels_lines.axes_manager[1].units = "eV"
print(eels_lines)
print(eels_lines.axes_manager)

In [ ]:
%matplotlib qt
eels_lines = eels_lines.remove_background(zero_fill=True, fast=False, show_progressbar=True, signal_range=(2.0, 2.5))

In [ ]:
eels_lines.data = eels_lines.data / np.max(eels_lines.data, axis=1)[:, np.newaxis]
print(np.max(eels_lines.data, axis=1))

In [ ]:
color_rep = ["black", "gray", "green", "red", "yellow", "blue", "purple", "orange", "lime", "cyan", "magenta", "lightgray", "peru", "springgreen", "deepskyblue", "hotpink"]
print(len(color_rep))

In [ ]:
def bandgap_measurement(spectrum, bg_range, slope_range):
    bg_left = int(np.round((bg_range[0]-e_start)/e_step))
    bg_right = int(np.round((bg_range[1]-e_start)/e_step))
    slope_left = int(np.round((slope_range[0]-e_start)/e_step))
    slope_right = int(np.round((slope_range[1]-e_start)/e_step))
    
    bg_flat = np.median(spectrum[bg_left:bg_right])
    poly_coeff = np.polyfit(energy_range[slope_left:slope_right], spectrum[slope_left:slope_right], deg=1)
    slope = poly_coeff[0]*energy_range + poly_coeff[1]
    
    intersection = (bg_flat - poly_coeff[1]) / poly_coeff[0]
    
    return intersection

In [ ]:
# line by line horizontally
%matplotlib qt
fig, ax = plt.subplots(1, 2, figsize=(10, 8.0))
ax[1].tick_params(axis="x", labelsize=15)
ax[1].axes.get_yaxis().set_visible(False)

bandgap = []
def line_range(line_s, line_e, interval, blur_sigma):
    ax[0].cla()
    ax[1].cla()
    
    del bandgap[:]
    image = np.sum(eelsi_binned.data, axis=2)
    ax[0].imshow(image, cmap="gray")
    ax[0].axis("off")
    mask = np.zeros(image.shape)
    mask[line_s:line_e] = np.max(image)
    ax[0].imshow(mask, cmap="Reds", alpha=0.2)
    
    for i in range(line_s, line_e, 1):
        blurred = ndimage.gaussian_filter(eels_lines.data[i], sigma=blur_sigma)
        Eg = bandgap_measurement(blurred, [1.0, 2.5], [3.0, 3.5])
        bandgap.append(Eg)
    
    #print(bandgap)
    ax[1].scatter(bandgap, np.arange(-line_s, -line_e, -1), color="r", s=5)
    ax[1].plot(bandgap, np.arange(-line_s, -line_e, -1), color="r")
    
    ax[1].grid()
    ax[1].set_xlim(2.9, 3.3)
    fig.canvas.draw()
    fig.tight_layout()
    
st = {"description_width":"initial"}    
ls_widget = pyw.IntText(value=0, description="index of the start line", style=st)
le_widget = pyw.IntText(value=eels_lines.data.shape[0], description="index of the end line", style=st)
i_widget = pyw.FloatText(value=0.2, description="adjustment for the interval between lines", style=st)
s_widget = pyw.FloatText(value=3.0, description="sigma for Gaussian blurring", style=st)

pyw.interact(line_range, line_s=ls_widget, line_e=le_widget, interval=i_widget, blur_sigma=s_widget)
plt.show()

In [ ]:
blur_bandgap = ndimage.gaussian_filter(bandgap, sigma=3)

fig, ax = plt.subplots(1, 1, figsize=(3, 10))
ax.scatter(blur_bandgap, np.arange(0, -165, -1), color="r", s=5)
ax.plot(blur_bandgap, np.arange(0, -165, -1), color="r")
ax.axes.get_yaxis().set_visible(False)
ax.grid()
fig.tight_layout()
plt.show()

In [ ]:
# line by line vertically
%matplotlib qt
fig, ax = plt.subplots(1, 2, figsize=(12, 5.0))
ax[1].tick_params(axis="x", labelsize=15)
ax[1].axes.get_xaxis().set_visible(False)

bandgap = []
def line_range(line_s, line_e, interval, blur_sigma):
    ax[0].cla()
    ax[1].cla()
    
    del bandgap[:]
    image = np.sum(eelsi_binned.data, axis=2)
    ax[0].imshow(image, cmap="gray")
    ax[0].axis("off")
    mask = np.zeros(image.shape)
    mask[:, line_s:line_e] = np.max(image)
    ax[0].imshow(mask, cmap="Reds", alpha=0.2)
    
    for i in range(line_s, line_e, 1):
        blurred = ndimage.gaussian_filter(eels_lines.data[i], sigma=blur_sigma)
        Eg = bandgap_measurement(blurred, [1.0, 2.5], [3.2, 3.6])
        bandgap.append(Eg)
    
    #print(bandgap)
    ax[1].scatter(np.arange(line_s, line_e, 1), bandgap, color="r", s=5)
    ax[1].plot(np.arange(line_s, line_e, 1), bandgap, color="r")
    
    ax[1].grid()
    ax[1].set_ylim(2.7, 3.3)
    fig.canvas.draw()
    fig.tight_layout()
    
st = {"description_width":"initial"}    
ls_widget = pyw.IntText(value=0, description="index of the start line", style=st)
le_widget = pyw.IntText(value=eels_lines.data.shape[0], description="index of the end line", style=st)
i_widget = pyw.FloatText(value=0.2, description="adjustment for the interval between lines", style=st)
s_widget = pyw.FloatText(value=3.0, description="sigma for Gaussian blurring", style=st)

pyw.interact(line_range, line_s=ls_widget, line_e=le_widget, interval=i_widget, blur_sigma=s_widget)
plt.show()

In [ ]:
# line by line horizontally
%matplotlib qt
fig, ax = plt.subplots(1, 4, figsize=(15, 4.5))
ax[1].tick_params(axis="x", labelsize=15)
ax[1].axes.get_yaxis().set_visible(False)
ax[2].axes.get_yaxis().set_visible(False)
ax[3].axes.get_yaxis().set_visible(False)

def line_range(line_s, line_e, interval, blur_sigma):
    ax[0].cla()
    ax[1].cla()
    ax[2].cla()
    ax[3].cla()
    
    image = np.sum(eelsi_binned.data, axis=2)
    ax[0].imshow(image, cmap="gray")
    ax[0].axis("off")
    mask = np.zeros(image.shape)
    mask[line_s:line_e] = np.max(image)
    ax[0].imshow(mask, cmap="Reds", alpha=0.2)
    
    bandgap = []
    for i in range(line_s, line_e, 1):
        ax[2].plot(energy_range, eels_lines.data[i]-i*interval)
        blurred = ndimage.gaussian_filter(eels_lines.data[i], sigma=blur_sigma)
        Eg = bandgap_measurement(blurred, [1.0, 2.5], [3.5, 4.0])
        ax[3].plot(energy_range, blurred-i*interval, label="Eg = %.2f eV"%Eg)
        bandgap.append(Eg)
        
    ax[1].scatter(bandgap, np.arange(-line_s, -line_e, -1), color="r", s=5)
    ax[1].plot(bandgap, np.arange(-line_s, -line_e, -1), color="r")
    
    ax[1].grid()
    ax[2].grid()
    ax[3].grid()
    #ax[2].legend()
    fig.canvas.draw()
    fig.tight_layout()
    
st = {"description_width":"initial"}    
ls_widget = pyw.IntText(value=0, description="index of the start line", style=st)
le_widget = pyw.IntText(value=eels_lines.data.shape[0], description="index of the end line", style=st)
i_widget = pyw.FloatText(value=0.2, description="adjustment for the interval between lines", style=st)
s_widget = pyw.FloatText(value=3.0, description="sigma for Gaussian blurring", style=st)

pyw.interact(line_range, line_s=ls_widget, line_e=le_widget, interval=i_widget, blur_sigma=s_widget)
plt.show()

In [ ]:
%matplotlib qt
fig, ax = plt.subplots(1, 1, figsize=(5, 15))
for i in range(len(eels_lines.data)):
    ax.plot(energy_range, eels_lines.data[i]+i*0.3)

ax.grid()
ax.tick_params(axis="both", labelsize=10)
ax.set_facecolor("black")
fig.tight_layout()
plt.show()